# <img style="float: left; padding-right: 20px; height: 70px" src="https://i.imgur.com/cMzxwTN.jpg"> Fundamentos de Machine Learning
## Sesion 2, Actividad II

**Universidad Privada Boliviana**<br/>
**Diplomado en Machine Learning y Ciencia de Datos**<br/>


In [ ]:
# Import necessary libraries
%matplotlib inline
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


### 1. Leer el dataset

In [ ]:
# Read the file "dataset.csv" as a Pandas dataframe
df = pd.read_csv("dataset.csv")

# Eche un vistazo rápido al conjunto de datos
df.head()


In [ ]:
# Asigne los valores de las columnas x e y a la
# variables predictoras y de respuesta
x = df[["x"]].values
y = df.y.values


### 2. Dividir en Train-validation sets

In [ ]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y validación con un conjunto de entrenamiento del 75 %
# Establecer estado_aleatorio=1
x_train, x_val, y_train, y_val = train_test_split(x,y,train_size=0.75,random_state=0)


### 3. Realizar la validacion cruzada con 4 pliegues con un polinomio de primer grado

In [ ]:
### codigo aca


# obtener el error de validacion cruzada 

#### 3.1 Analizar la variable de validacion cruzada


In [ ]:
### Encontrar el error promedio de validacion cruzada

### 4. Encontrar el  MSE para polinomios de grado variable

In [ ]:
# Seleccionar el grado maximo a iterar
maxdeg = 15
training_error = []
validation_error = []
cross_validation_error = []
for d in range(maxdeg):
    # calculo de los polynomial features
    x_poly = ...
    
    x_poly_train = ...
    x_poly_val   = ...
    # Inicializar el modelo de regresion sin utilizar el intercept
    lreg = ...
    # Ajustar el modelo en los datos de entrenamiento
    lreg.fit(x_poly_train,y_train)

    # Utilizar el modelo entrenado para predecir en los datos transformados del train set
    y_train_pred = lreg.predict(x_poly_train)
    
    # Utilizar el modelo entrenado para predecir en los datos transformados del validation set
    y_val_pred   = lreg.predict(x_poly_val)

    # Calcular el error en base a los datos en y_train_pred y y_val_pred
    training_error.append(mean_squared_error(y_train,y_train_pred))
    validation_error.append(mean_squared_error(y_val,y_val_pred))

    # realizar cross validation 
    mse_score = ...
    cross_validation_error.append(...)


    

#### 4.1 Encontrar el grado optimo del polinomio

In [ ]:
...

#### 4.2 Graficar los errores obtenidos

In [ ]:

fig,ax = plt.subplots(1,2,figsize=(16,8))

# Graficar el error del training set con etiquetas
ax[0].plot(range(maxdeg),np.log(training_error),label="Training error")
# Graficar el error del validation set con etiquetas
ax[0].plot(range(maxdeg),np.log(validation_error),label="Validation error")

# Graficar el error del training set con etiquetas
ax[1].plot(range(maxdeg),np.log(training_error),label="Training error")
# Graficar el error del validation set con etiquetas
ax[1].plot(range(maxdeg),np.log(cross_validation_error),label="Cross-Validation error")


# agregar etiquetas a los ejes:
ax[0].set_xlabel("Grado del polinomio")
ax[0].set_ylabel("Log MSE")
ax[0].set_title("Log de error de validacion vs grado")
ax[0].legend(loc='best')

# agregar etiquetas a los ejes:
ax[1].set_xlabel("Grado del polinomio")
ax[1].set_ylabel("Log MSE")
ax[1].set_title("Log de Cross Val. error vs grado")
ax[1].legend(loc='best')

plt.show()

⏸ Si ejecutas el ejercicio con un estado aleatorio de 0, ¿notas algún cambio? ¿Qué conclusión puedes sacar de este experimento?


In [ ]:

answer1 = '---'


### 5. Analisis para otro dataset

Repetir los pasos anteriores $1,2,4$ para el `dataset1.csv`


Anotar sus observaciones